In [30]:
from dotenv import load_dotenv
import os

load_dotenv(verbose=True)
key = os.getenv('OPENAI_API_KEY')

**출력 파서(Output parser)** 는 언어 모델(LLM)의 출력을 더 유용하고 구조화된 형태로 변환하는 컴포넌트입니다. <br>
파서는 `해석하다` 라는 의미를 가지고 있는데, LLM이 준 답변을 구조화된 형태로 변환해 주는 것이 파서의 역할입니다.

답변을 받을 스키마를 파서에 정의하고 LLM에 붙여서 사용합니다. <br>
LLM의 출력을 받아서 구조화된 데이터 형식으로 변환 해주는 역할을 합니다.

## **PydanticOutputParser**

`PydanticOutputParser` 는 언어 모델의 출력을 더 구조화된 정보로 변환하는 데 도움이 되는 클래스입니다.<br>
단순 텍스트 형태의 응답 대신, 사용자가 필요로 하는 정보를 명확하고 체계적인 형태로 제공할 수 있습니다.

`PydanticOutputParser` 에는 주로 두 가지 핵심 메소드가 구현되어야 합니다. <br>
- `get_format_instructions()` : 
    - 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침(instruction)을 제공합니다. 
    - 예를 들어, 언어 모델이 출력해야 할 데이터의 필드와 그 형태를 설명하는 지침을 문자열로 반환할 수 있습니다.

- `parse()` :
    - 언어 모델의 출력(문자열로 가정)을 받아들여 이를 특정 구조로 분석하고 변환합니다. 
    - ydantic와 같은 도구를 사용하여, 입력된 문자열을 사전 정의된 스키마에 따라 검증하고, 해당 스키마를 따르는 데이터 구조로 변환합니다. 구조화된 객체로 변환해 줍니다.

In [35]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

In [36]:
email_conversation = """From: 이인환 (yoogane_kmu@naver.com)
To: Korea AI 팀장님 (korea_ai@naver.com)
Subject: RAG 솔루션 시연 관련 미팅 제안

안녕하세요, Korea AI 팀장님,

저는 소프트웨어놀이터의 이인환입니다. 최근 귀사에서 AI를 활용한 혁신적인 솔루션을 모색 중이라는 소식을 들었습니다. 

소프트웨어놀이터는 AI 및 RAG 솔루션 분야에서 다양한 경험과 노하우를 가진 기업으로, 귀사의 요구에 맞는 최적의 솔루션을 제공할 수 있다고 자부합니다.
저희 소프트웨어놀이터의 RAG 솔루션은 귀사의 데이터 활용을 극대화하고, 실시간으로 정확한 정보 제공을 통해 비즈니스 의사결정을 지원하는 데 탁월한 성능을 보입니다. 이 솔루션은 특히 다양한 산업에서의 성공적인 적용 사례를 통해 그 효과를 입증하였습니다.

귀사와의 협력 가능성을 논의하고, 저희 RAG 솔루션의 구체적인 기능과 적용 방안을 시연하기 위해 미팅을 제안드립니다. 

다음 주 월요일(12월 9일) 오후 3시에 귀사 사무실에서 만나 뵐 수 있을까요?
미팅 시간을 조율하기 어려우시다면, 편하신 다른 일정을 알려주시면 감사하겠습니다. Korea AI 팀장님과의 소중한 만남을 통해 상호 발전적인 논의가 이루어지길 기대합니다.

감사합니다.

이인환

소프트웨어놀이터 AI 솔루션팀
"""

In [37]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")

In [38]:
parser = PydanticOutputParser(pydantic_object=EmailSummary)     # PydanticOutputParser 생성

In [39]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [40]:
# question : 사용자 질문
# email_conversation : 이메일의 본문을 입력합니다.
# format: 형식을 지정합니다.

In [41]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}          

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

In [42]:
# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())

In [43]:
llm = ChatOpenAI(api_key=key, model='gpt-4o-mini')

In [44]:
chain = prompt | llm

In [45]:
response = chain.invoke(
    {
        "email_conversation": email_conversation,
        "question": "이메일 내용중 주요 내용을 추출해 주세요.",
    }
)

In [46]:
print(response.content)

```json
{
  "person": "이인환",
  "email": "yoogane_kmu@naver.com",
  "subject": "RAG 솔루션 시연 관련 미팅 제안",
  "summary": "소프트웨어놀이터의 이인환이 Korea AI 팀장님에게 RAG 솔루션의 시연을 위한 미팅을 제안하며, 귀사의 데이터 활용을 극대화하고 비즈니스 의사결정을 지원할 수 있는 솔루션을 제공할 수 있다고 언급함.",
  "date": "12월 9일 오후 3시"
}
```


In [47]:
structured_output = parser.parse(response.content)  # 객체로 만들기

In [48]:
print(structured_output)

person='이인환' email='yoogane_kmu@naver.com' subject='RAG 솔루션 시연 관련 미팅 제안' summary='소프트웨어놀이터의 이인환이 Korea AI 팀장님에게 RAG 솔루션의 시연을 위한 미팅을 제안하며, 귀사의 데이터 활용을 극대화하고 비즈니스 의사결정을 지원할 수 있는 솔루션을 제공할 수 있다고 언급함.' date='12월 9일 오후 3시'
